## 导入库

In [1]:
import os
import json
import math
import random
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook as tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F

## 生成数据

+ **数据格式说明**  
  数据集的格式为{graph1, graph2, ..., graphn}
  其中，graphi为第i个graph数据，graphi的数据结构为G_i(N, E)：
        N = {v_1, v_2, ..., v_n}  # n个节点集合
        E = {e_1, e_2, ..., e_t}  # t条边集合
   其中，e_j的数据结构为(v_p, v_q)，表示该边两端的节点。
+ **任务说明**  
  给定子图sub_graph，训练集{graph1, graph2, ..., graphn}，模型目标就是对其中的graph中的节点进行分类。  
  如果graphi包含sub_graph和节点v，且节点v属于sub_graph中的一个节点，那么节点v的标签为1，反之，标签为-1。模型就是要预测每个节点的标签是1还是-1。
+ **其它说明**  
  这里使用的数据直接按照论文《The Graph Neural Network Model》所述的子图匹配任务进行实现，节点的特征就是数字1-10  
    1, 2, 3, 4, 5, 6, 7, 8, 9, 10  
  因此，节点集合使用节点特征的列表表示，边集合中的每一条边(i,j)中的i和j表示该边两端的节点在节点列表中对应的索引。

In [2]:
'''
使用networkx库生成graph的图像，并显示，节点中显示的数字表示节点特征数字。
Input :
    graph : (Node_list, Edge_list)
'''
def graphFigure(graph):
    G = nx.Graph()
    node_feat_list, edge_list = graph
    node_names = ["v{}".format(i) for i in range(len(node_feat_list))]
    G.add_nodes_from(node_names, feat=node_feat_list)
    G.add_edges_from([("v{}".format(i),"v{}".format(j)) for i,j in edge_list])
    labels = dict([("v{}".format(i), feat) for i, feat in enumerate(node_feat_list)])
    plt.figure(0)
    nx.draw_spectral(G=G, 
                     node_size=1000,
                     node_color="g",
                     with_labels=True, 
                     font_weight='bold',
                     labels=labels)
'''
用于产生一个graph数据，首先随机生成若干个节点，然后随机对这些节点
进行连接，得到一个随机的graph，然后将subgraph插入产生的随机graph，
得到graph数据。
Input :
    sub_graph : 子图数据
    node_num : 生成随机graph的节点个数
    edge_num : 生成随机graph的边条数
    ins_edge_num : 将subgraph插入随机graph时，连接到随机graph的边条数
'''
def genGraph(sub_graph, node_num, edge_num, ins_edge_num):
    nodes_list = list(np.random.randint(low=1, high=11, size=(node_num)))
    label_list = [-1] * len(nodes_list)
    edges_list = []
    edge_proba = edge_num / (node_num * (node_num - 1) / 2)
    for n in range(node_num-1):
        end_nodes = np.random.choice(a=np.arange(n+1, node_num), 
                                     size=(round(edge_proba * (node_num-1-n))),
                                     replace=False)
        edges_list.extend([(n, e) for e in end_nodes])
    # Insert subgraph
    nodes_list.extend(sub_graph[0])  # add subgraph nodes
    label_list.extend([1]*len(sub_graph[0]))
    head_nodes = np.random.choice(a=np.arange(node_num),
                                  size=(ins_edge_num))
    tail_nodes = np.random.choice(a=np.arange(node_num, len(nodes_list)),
                                  size=(ins_edge_num))
    edges_list.extend([(i,j) for i,j in zip(head_nodes, tail_nodes)])
    edges_list.extend([(i+node_num,j+node_num) for i,j in sub_graph[1]])
    return (nodes_list, edges_list, label_list)
    
'''
Definition of subgraph and dataset generation
'''
np.random.seed(0)
N = 600          # graph dataset length
node_nums = [5, 10, 15, 20]
ins_nums = [4, 8, 12, 16]


sub_graph = ([1,5,5,8],[(0,1),(0,3),(1,3),(2,3)])  # (Node_list, Edge_list)
dataset = []
for i in range(N):
    nnum = random.choice(node_nums)   # node_num
    ins_en = random.choice(ins_nums)# ins_edge_num
    dataset.append(genGraph(sub_graph, nnum, 2*nnum, ins_en))

## 构建模型

### GCN构建

In [4]:
'''
同Linear GNN
'''
class AggrSum(nn.Module):
    def __init__(self):
        super(AggrSum, self).__init__()
    
    def forward(self, H, X_node, node_num):
        # H : (N, s) -> (V, s)
        # X_node : (N, )
        mask = torch.stack([X_node] * node_num, 0)
        mask = mask.float() - torch.unsqueeze(torch.range(0,node_num-1).float(), 1)
        mask = (mask == 0).float()
        # (V, N) * (N, s) -> (V, s)
        return torch.mm(mask, H)

'''
用于实现GCN的卷积块。
Initialize :
Input :
    in_channel : (int)输入的节点特征维度
    out_channel : (int)输出的节点特征维度
Forward :
Input :
    x : (Tensor)节点的特征矩阵，shape为(N, in_channel)，N为节点个数
    edge_index : (Tensor)边矩阵，shape为(2, E)，E为边个数。
Output :
    out : (Tensor)新的特征矩阵，shape为(N, out_channel)
'''
class GCNConv(nn.Module):
    def __init__(self, in_channel, out_channel):
        super(GCNConv, self).__init__()
        self.linear = nn.Linear(in_channel, out_channel)
        self.aggregation = AggrSum()
        
    def forward(self, x, edge_index):
        # Add self-connect edges
        edge_index = self.addSelfConnect(edge_index, x.shape[0])
        node_num = x.shape[0]
        
        # Apply linear transform
        x = self.linear(x)
        
        # Normalize message
        row, col = edge_index
        deg = self.calDegree(row, x.shape[0]).float()
        deg_sqrt = deg.pow(-0.5)  # (N, )
        norm = deg_sqrt[row] * deg_sqrt[col]
        
        # Node feature matrix
        tar_matrix = torch.index_select(x, dim=0, index=col)
        tar_matrix = norm.view(-1, 1) * tar_matrix  # (E, out_channel)
        # Aggregate information
        aggr =  self.aggregation(tar_matrix, row, node_num)  # (N, out_channel)
        return aggr
        
    def calDegree(self, edges, num_nodes):
        ind, deg = np.unique(edges.cpu().numpy(), return_counts=True)
        deg_tensor = torch.zeros((num_nodes, ), dtype=torch.long)
        deg_tensor[ind] = torch.from_numpy(deg)
        return deg_tensor.to(edges.device)
    
    def addSelfConnect(self, edge_index, num_nodes):
        selfconn = torch.stack([torch.range(0, num_nodes-1, dtype=torch.long)]*2,
                               dim=0).to(edge_index.device)
        return torch.cat(tensors=[edge_index, selfconn],
                         dim=1)

### 两层GCN构建

In [5]:
'''
构建模型，使用两层GCN，第一层GCN使得节点特征矩阵
    (N, in_channel) -> (N, out_channel)
第二层GCN直接输出
    (N, out_channel) -> (N, num_class)
激活函数使用relu函数，网络最后对节点的各个类别score使用softmax归一化。
'''
class Net(nn.Module):
    def __init__(self, feat_dim, num_class):
        super(Net, self).__init__()
        self.conv1 = GCNConv(feat_dim, 16)
        self.conv2 = GCNConv(16, num_class)
    
    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        
        return F.softmax(x, dim=-1)

### 准备输入数据

In [6]:
embedding = np.diag(np.ones((10)))

'''
得到x和edge_index输入
'''
def getInput(graph):
    x = embedding[[i-1 for i in graph[0]]]
    edge_index = np.array([np.array([i,j]) for i,j in graph[1]]).T  # (2, E)
    edge_index = np.concatenate([edge_index] * 2, axis=1)   # (2, 2*E)
    y = np.array(graph[2])
    return x, edge_index, y

### 开始训练

In [ ]:
'''
开始训练模型
'''
device = torch.device('cpu')# torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net(10, 2).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
# loss_fn = nn.CrossEntropyLoss()

'''
模型评估函数
'''
def evalModel(model, dataset):
    for graph in dataset:
        x, edge_index, y = getInput(graph)
        x = torch.from_numpy(x).float()
        edge_index = torch.from_numpy(edge_index).long()
        y[y < 0] = 0
        y = torch.from_numpy(y).long()
        
        acc_list = []
        _, pred = model(x, edge_index).max(dim=1)
        acc_list.append(float(pred.eq(y).sum().item())/y.shape[0])
    return sum(acc_list)/ len(acc_list)

for epoch in range(200):
    for step, graph in enumerate(dataset[:400]):
        # Get input
        x, edge_index, y = getInput(graph)
        x = torch.from_numpy(x).float().to(device)
        edge_index = torch.from_numpy(edge_index).long().to(device)
        y[y < 0] = 0
        y = torch.from_numpy(y).long().to(device)
        
        model.train()
        optimizer.zero_grad()

        # Get output
        out = model(x, edge_index)   # (N, 2)
        # Get loss
        loss = F.cross_entropy(out, y)

        # Backward
        loss.backward()
        optimizer.step()
        
        # Get predictions and calculate training accuracy
        _, pred = out.cpu().detach().max(dim=-1)  # (N)
        y = y.cpu().detach()
        correct = float(pred.eq(y).sum().item())
        acc = correct / pred.shape[0]
        print('[Epoch {}/200, step {}/400] Loss {:.4f}, train acc {:.4f}'.format(epoch, step, loss.cpu().detach().data.item(), acc))

    # Evaluation on test data every 10 epochs
    if (epoch+1) % 10 == 0:
        model.eval()
        print('Accuracy: {:.4f}'.format(evalModel(model, dataset[400:])))

D:\anaconda\lib\site-packages\ipykernel_launcher.py:64: UserWarning: torch.range is deprecated in favor of torch.arange and will be removed in 0.5. Note that arange generates values in [start; end), not [start; end].
D:\anaconda\lib\site-packages\ipykernel_launcher.py:12: UserWarning: torch.range is deprecated in favor of torch.arange and will be removed in 0.5. Note that arange generates values in [start; end), not [start; end].
  if sys.path[0] == '':


[Epoch 0/200, step 0/400] Loss 0.7414, train acc 0.2105
[Epoch 0/200, step 1/400] Loss 0.7127, train acc 0.2857
[Epoch 0/200, step 2/400] Loss 0.7048, train acc 0.4444
[Epoch 0/200, step 3/400] Loss 0.7072, train acc 0.4444
[Epoch 0/200, step 4/400] Loss 0.7023, train acc 0.1429
[Epoch 0/200, step 5/400] Loss 0.6736, train acc 0.8421
[Epoch 0/200, step 6/400] Loss 0.6765, train acc 0.7143
[Epoch 0/200, step 7/400] Loss 0.6474, train acc 0.7500
[Epoch 0/200, step 8/400] Loss 0.6580, train acc 0.7895
[Epoch 0/200, step 9/400] Loss 0.6420, train acc 0.7917
[Epoch 0/200, step 10/400] Loss 0.6411, train acc 0.5556
[Epoch 0/200, step 11/400] Loss 0.6718, train acc 0.5556
[Epoch 0/200, step 12/400] Loss 0.6423, train acc 0.7143
[Epoch 0/200, step 13/400] Loss 0.6470, train acc 0.7895
[Epoch 0/200, step 14/400] Loss 0.5727, train acc 0.8333
[Epoch 0/200, step 15/400] Loss 0.5860, train acc 0.7895
[Epoch 0/200, step 16/400] Loss 0.5672, train acc 0.8333
[Epoch 0/200, step 17/400] Loss 0.5922, t

[Epoch 0/200, step 284/400] Loss 0.3269, train acc 1.0000
[Epoch 0/200, step 285/400] Loss 0.3197, train acc 1.0000
[Epoch 0/200, step 286/400] Loss 0.4499, train acc 0.8421
[Epoch 0/200, step 287/400] Loss 0.4250, train acc 0.8889
[Epoch 0/200, step 288/400] Loss 0.3708, train acc 0.9474
[Epoch 0/200, step 289/400] Loss 0.4346, train acc 0.8947
[Epoch 0/200, step 290/400] Loss 0.3156, train acc 1.0000
[Epoch 0/200, step 291/400] Loss 0.3751, train acc 0.9286
[Epoch 0/200, step 292/400] Loss 0.4194, train acc 0.9286
[Epoch 0/200, step 293/400] Loss 0.3801, train acc 0.9474
[Epoch 0/200, step 294/400] Loss 0.3875, train acc 0.9583
[Epoch 0/200, step 295/400] Loss 0.3646, train acc 1.0000
[Epoch 0/200, step 296/400] Loss 0.3314, train acc 1.0000
[Epoch 0/200, step 297/400] Loss 0.3341, train acc 1.0000
[Epoch 0/200, step 298/400] Loss 0.3210, train acc 1.0000
[Epoch 0/200, step 299/400] Loss 0.3350, train acc 1.0000
[Epoch 0/200, step 300/400] Loss 0.3816, train acc 0.9167
[Epoch 0/200, 

[Epoch 1/200, step 167/400] Loss 0.3480, train acc 1.0000
[Epoch 1/200, step 168/400] Loss 0.3275, train acc 1.0000
[Epoch 1/200, step 169/400] Loss 0.3524, train acc 0.9583
[Epoch 1/200, step 170/400] Loss 0.3473, train acc 1.0000
[Epoch 1/200, step 171/400] Loss 0.5280, train acc 0.7895
[Epoch 1/200, step 172/400] Loss 0.3170, train acc 1.0000
[Epoch 1/200, step 173/400] Loss 0.3819, train acc 0.9286
[Epoch 1/200, step 174/400] Loss 0.3277, train acc 1.0000
[Epoch 1/200, step 175/400] Loss 0.3255, train acc 1.0000
[Epoch 1/200, step 176/400] Loss 0.3173, train acc 1.0000
[Epoch 1/200, step 177/400] Loss 0.3932, train acc 0.9286
[Epoch 1/200, step 178/400] Loss 0.3178, train acc 1.0000
[Epoch 1/200, step 179/400] Loss 0.3166, train acc 1.0000
[Epoch 1/200, step 180/400] Loss 0.3138, train acc 1.0000
[Epoch 1/200, step 181/400] Loss 0.3396, train acc 1.0000
[Epoch 1/200, step 182/400] Loss 0.4163, train acc 0.8947
[Epoch 1/200, step 183/400] Loss 0.3328, train acc 1.0000
[Epoch 1/200, 

[Epoch 2/200, step 50/400] Loss 0.3163, train acc 1.0000
[Epoch 2/200, step 51/400] Loss 0.3174, train acc 1.0000
[Epoch 2/200, step 52/400] Loss 0.4449, train acc 0.7778
[Epoch 2/200, step 53/400] Loss 0.5507, train acc 0.7143
[Epoch 2/200, step 54/400] Loss 0.3225, train acc 1.0000
[Epoch 2/200, step 55/400] Loss 0.3139, train acc 1.0000
[Epoch 2/200, step 56/400] Loss 0.3176, train acc 1.0000
[Epoch 2/200, step 57/400] Loss 0.3521, train acc 0.9583
[Epoch 2/200, step 58/400] Loss 0.3425, train acc 1.0000
[Epoch 2/200, step 59/400] Loss 0.4187, train acc 0.8947
[Epoch 2/200, step 60/400] Loss 0.3196, train acc 1.0000
[Epoch 2/200, step 61/400] Loss 0.3441, train acc 1.0000
[Epoch 2/200, step 62/400] Loss 0.3311, train acc 1.0000
[Epoch 2/200, step 63/400] Loss 0.3225, train acc 1.0000
[Epoch 2/200, step 64/400] Loss 0.3187, train acc 1.0000
[Epoch 2/200, step 65/400] Loss 0.3271, train acc 1.0000
[Epoch 2/200, step 66/400] Loss 0.3164, train acc 1.0000
[Epoch 2/200, step 67/400] Loss

...

[Epoch 32/200, step 51/400] Loss 0.3159, train acc 1.0000
[Epoch 32/200, step 52/400] Loss 0.4193, train acc 0.8889
[Epoch 32/200, step 53/400] Loss 0.4877, train acc 0.7857
[Epoch 32/200, step 54/400] Loss 0.3165, train acc 1.0000
[Epoch 32/200, step 55/400] Loss 0.3147, train acc 1.0000
[Epoch 32/200, step 56/400] Loss 0.3164, train acc 1.0000
[Epoch 32/200, step 57/400] Loss 0.3536, train acc 0.9583
[Epoch 32/200, step 58/400] Loss 0.3193, train acc 1.0000
[Epoch 32/200, step 59/400] Loss 0.3974, train acc 0.8947
[Epoch 32/200, step 60/400] Loss 0.3160, train acc 1.0000
[Epoch 32/200, step 61/400] Loss 0.3226, train acc 1.0000
[Epoch 32/200, step 62/400] Loss 0.3316, train acc 1.0000
[Epoch 32/200, step 63/400] Loss 0.3488, train acc 1.0000
[Epoch 32/200, step 64/400] Loss 0.3247, train acc 1.0000
[Epoch 32/200, step 65/400] Loss 0.3215, train acc 1.0000
[Epoch 32/200, step 66/400] Loss 0.3518, train acc 0.9286
[Epoch 32/200, step 67/400] Loss 0.5011, train acc 0.7778
[Epoch 32/200,

[Epoch 32/200, step 328/400] Loss 0.3231, train acc 1.0000
[Epoch 32/200, step 329/400] Loss 0.3219, train acc 1.0000
[Epoch 32/200, step 330/400] Loss 0.3229, train acc 1.0000
[Epoch 32/200, step 331/400] Loss 0.3171, train acc 1.0000
[Epoch 32/200, step 332/400] Loss 0.3825, train acc 0.9286
[Epoch 32/200, step 333/400] Loss 0.3154, train acc 1.0000
[Epoch 32/200, step 334/400] Loss 0.3672, train acc 0.9474
[Epoch 32/200, step 335/400] Loss 0.3163, train acc 1.0000
[Epoch 32/200, step 336/400] Loss 0.3159, train acc 1.0000
[Epoch 32/200, step 337/400] Loss 0.4578, train acc 0.8571
[Epoch 32/200, step 338/400] Loss 0.3147, train acc 1.0000
[Epoch 32/200, step 339/400] Loss 0.3169, train acc 1.0000
[Epoch 32/200, step 340/400] Loss 0.3173, train acc 1.0000
[Epoch 32/200, step 341/400] Loss 0.3222, train acc 1.0000
[Epoch 32/200, step 342/400] Loss 0.5070, train acc 0.7917
[Epoch 32/200, step 343/400] Loss 0.3247, train acc 1.0000
[Epoch 32/200, step 344/400] Loss 0.3148, train acc 1.00

[Epoch 33/200, step 207/400] Loss 0.3220, train acc 1.0000
[Epoch 33/200, step 208/400] Loss 0.3148, train acc 1.0000
[Epoch 33/200, step 209/400] Loss 0.3145, train acc 1.0000
[Epoch 33/200, step 210/400] Loss 0.3201, train acc 1.0000
[Epoch 33/200, step 211/400] Loss 0.3566, train acc 0.9583
[Epoch 33/200, step 212/400] Loss 0.3521, train acc 1.0000
[Epoch 33/200, step 213/400] Loss 0.3181, train acc 1.0000
[Epoch 33/200, step 214/400] Loss 0.3165, train acc 1.0000
[Epoch 33/200, step 215/400] Loss 0.3214, train acc 1.0000
[Epoch 33/200, step 216/400] Loss 0.3157, train acc 1.0000
[Epoch 33/200, step 217/400] Loss 0.4613, train acc 0.8421
[Epoch 33/200, step 218/400] Loss 0.3172, train acc 1.0000
[Epoch 33/200, step 219/400] Loss 0.3266, train acc 1.0000
[Epoch 33/200, step 220/400] Loss 0.3292, train acc 1.0000
[Epoch 33/200, step 221/400] Loss 0.3154, train acc 1.0000
[Epoch 33/200, step 222/400] Loss 0.3610, train acc 0.9286
[Epoch 33/200, step 223/400] Loss 0.3220, train acc 1.00

[Epoch 34/200, step 85/400] Loss 0.3244, train acc 1.0000
[Epoch 34/200, step 86/400] Loss 0.3380, train acc 1.0000
[Epoch 34/200, step 87/400] Loss 0.3168, train acc 1.0000
[Epoch 34/200, step 88/400] Loss 0.5166, train acc 0.7895
[Epoch 34/200, step 89/400] Loss 0.3421, train acc 1.0000
[Epoch 34/200, step 90/400] Loss 0.3191, train acc 1.0000
[Epoch 34/200, step 91/400] Loss 0.3178, train acc 1.0000
[Epoch 34/200, step 92/400] Loss 0.3246, train acc 1.0000
[Epoch 34/200, step 93/400] Loss 0.3358, train acc 1.0000
[Epoch 34/200, step 94/400] Loss 0.4132, train acc 0.9286
[Epoch 34/200, step 95/400] Loss 0.3684, train acc 0.9474
[Epoch 34/200, step 96/400] Loss 0.4950, train acc 0.7917
[Epoch 34/200, step 97/400] Loss 0.3264, train acc 1.0000
[Epoch 34/200, step 98/400] Loss 0.3274, train acc 1.0000
[Epoch 34/200, step 99/400] Loss 0.3392, train acc 1.0000
[Epoch 34/200, step 100/400] Loss 0.3346, train acc 1.0000
[Epoch 34/200, step 101/400] Loss 0.3944, train acc 0.8947
[Epoch 34/20

[Epoch 34/200, step 362/400] Loss 0.3408, train acc 1.0000
[Epoch 34/200, step 363/400] Loss 0.3432, train acc 0.9474
[Epoch 34/200, step 364/400] Loss 0.5264, train acc 0.7778
[Epoch 34/200, step 365/400] Loss 0.3978, train acc 0.9167
[Epoch 34/200, step 366/400] Loss 0.4247, train acc 0.8750
[Epoch 34/200, step 367/400] Loss 0.4232, train acc 0.8750
[Epoch 34/200, step 368/400] Loss 0.3387, train acc 1.0000
[Epoch 34/200, step 369/400] Loss 0.3709, train acc 1.0000
[Epoch 34/200, step 370/400] Loss 0.3348, train acc 1.0000
[Epoch 34/200, step 371/400] Loss 0.3789, train acc 0.8889
[Epoch 34/200, step 372/400] Loss 0.5320, train acc 0.7143
[Epoch 34/200, step 373/400] Loss 0.3205, train acc 1.0000
[Epoch 34/200, step 374/400] Loss 0.3180, train acc 1.0000
[Epoch 34/200, step 375/400] Loss 0.4014, train acc 0.9286
[Epoch 34/200, step 376/400] Loss 0.3191, train acc 1.0000
[Epoch 34/200, step 377/400] Loss 0.3311, train acc 1.0000
[Epoch 34/200, step 378/400] Loss 0.3241, train acc 1.00